# RFSoC Limitations

This notebook outlines the limitations of the RFSoC and the reasons why different methods of pulse generation may be utilised. 

Board parameters are stored in the dictionary `soccfg._cfg`.

In [53]:
# Board libraries
from qick import *
import Pyro4

# Pyro communicates with board
Pyro4.config.SERIALIZER = "pickle"
Pyro4.config.PICKLE_PROTOCOL_VERSION=4

# Static IP proxy
ns_host = "172.24.106.21" # <-- SET IP ADDRESS
ns_port = 8888
proxy_name = "myqick"

# QICK object
ns = Pyro4.locateNS(host=ns_host, port=ns_port)
soc = Pyro4.Proxy(ns.lookup(proxy_name))
soccfg = QickConfig(soc.get_cfg())
# print(soccfg)

## DAC and DIG Pulses

### tproc

The time at which DAC and DIG pulses start, and DIG pulses end, is dictated by the QICK timed-processor (tproc).
- Advantages:
    - Synchronisation between channels.
    - Easy to configure.
- Disadvantage:
    - Significant discretisation of time at which pulses may start.

### DAC Fabric

The length of DAC pulses when using a `"const"` envelope is dictated by the DAC fabric frequency.
- Advantage:
  - Easy to control amplitude and phase of DAC channels.
- Disadvantage:
    - Significant discretisation of lengths of pulses.

### DAC Sampling Frequency

The length of DAC pulses when using `"arb"` envelope is dictated by the DAC sampling frequency. This is 16 times the DAC fabric frequency and these envelopes must be padded to have a length that is a multiple of 16.
- Advantages:
    - Precise control of I and Q amplitudes within DAC.
    - May create very short pulses, or pulses with unusual envelopes.
- Disadvantage:
    - More difficult to configure as 'raw' IQ amplitude envelopes utilised.

In [55]:
dac_ch = 0
f_tproc = soccfg['tprocs'][0]['f_time']
f_fabric = soccfg['gens'][dac_ch]['f_fabric']
fs = soccfg['gens'][dac_ch]['fs']

print(f"tproc frequency: {f_tproc} MHz")
print(f"--> DAC and DIG pulse start time discretisation: {round((1/f_tproc)*1e3,3)} ns")

print(f"\nDAC fabric frequency: {f_fabric} MHz")
print(f"DAC sampling frequency: {fs} MHz")
print(f"--> DAC pulse length discretisation (1 DAC fabric cycle): {round((1/f_fabric)*1e3,3)} ns")
print(f"--> Minimum 'arb' pulse length (1 DAC sampling cycle): {round(1/fs*1e6,3)} ps")
print(f"--> Minimum 'const' pulse length (2^2 - 1 DAC fabric cycles): {round(3*(1/f_fabric)*1e3,3)} ns")
print(f"--> Maximum pulse length (2^16 - 1 DAC fabric cycles): {round(65535*(1/f_fabric),3)} us")

tproc frequency: 409.6 MHz
--> DAC and DIG pulse start time discretisation: 2.441 ns

DAC fabric frequency: 614.4 MHz
DAC sampling frequency: 9830.4 MHz
--> DAC pulse length discretisation (1 DAC fabric cycle): 1.628 ns
--> Minimum 'arb' pulse length (1 DAC sampling cycle): 101.725 ps
--> Minimum 'const' pulse length (2^2 - 1 DAC fabric cycles): 4.883 ns
--> Maximum pulse length (2^16 - 1 DAC fabric cycles): 106.665 us


## ADC Readout Buffers

### Readout Buffer

- The built-in readout buffers of each channel are the most convenient to use.
- The `outsel` parameter of the `set_readout_registers()` function may be used to configure whether raw samples are taken or IQ demodulation is to be used.

### DDR4 Buffer

- The DDR4 buffer is used when many samples wish to be taken. It has by far the largest buffer period, however, it also downsamples the data by a factor of 8.

### MR Buffer

- The multi-rate buffer is used when the full-sampling frequency of the ADCs is to be used. It has the same buffer period as the readout buffers.

In [54]:
adc_ch = 0
fs_adc = soccfg['readouts'][adc_ch]['fs']
ro_buf_len = soccfg['readouts'][adc_ch]['buf_maxlen']
ddr4_buf_len = soccfg['ddr4_buf']['maxlen'] - soccfg['ddr4_buf']['junk_len']
mr_buf_len = soccfg['mr_buf']['maxlen']-soccfg['mr_buf']['junk_len']

print(f"ADC sampling frequency: {fs_adc} MHz")
print(f"--> Readout and DDR4 buffer (decimated) sampling frequencies: {fs_adc/8} MHz")
print(f"--> Multi-rate (MR) buffer sampling frequency: {fs_adc} MHz")

print(f"\nReadout buffer length: {ro_buf_len} samples")
print(f"--> Readout buffer period: {round((8/fs_adc)*ro_buf_len,3)} us")

print(f"\nDDR4 buffer length: {ddr4_buf_len} samples")
print(f"--> DDR4 buffer period: {round((8/fs_adc)*ddr4_buf_len/1e6,3)} s")

print(f"\nMR buffer length: {mr_buf_len} samples")
print(f"--> MR buffer period: {round((1/fs_adc)*mr_buf_len,3)} us")

# for key, value in soccfg._cfg.items():
#     print(key, value)

ADC sampling frequency: 4423.68 MHz
--> Readout and DDR4 buffer (decimated) sampling frequencies: 552.96 MHz
--> Multi-rate (MR) buffer sampling frequency: 4423.68 MHz

Readout buffer length: 1024 samples
--> Readout buffer period: 1.852 us

DDR4 buffer length: 1073741023 samples
--> DDR4 buffer period: 1.942 s

MR buffer length: 8184 samples
--> MR buffer period: 1.85 us
